In [ ]:
import cv2
import pandas as pd
from ultralytics import YOLO
from tracker import*

class Tracker:
    def __init__(self):
        # Store the center positions of the objects
        self.center_points = {}
        # Keep the count of the IDs
        # each time a new object id detected, the count will increase by one
        self.id_count = 0

    def update(self, objects_rect):
        # Objects boxes and ids
        objects_bbs_ids = []

        # Get center point of new object
        for rect in objects_rect:
            if len(rect) >= 4:  # Ensure rect has at least 4 values (x, y, w, h)
                x, y, w, h = rect[:4]  # Extract only the first 4 values
                cx = (x + x + w) // 2
                cy = (y + y + h) // 2

                # Find out if that object was detected already
                same_object_detected = False
                for id, pt in self.center_points.items():
                    dist = math.hypot(cx - pt[0], cy - pt[1])

                    if dist < 35:
                        self.center_points[id] = (cx, cy)
                        objects_bbs_ids.append([x, y, w, h, id])
                        same_object_detected = True
                        break

                # New object is detected we assign the ID to that object
                if not same_object_detected:
                    self.center_points[self.id_count] = (cx, cy)
                    objects_bbs_ids.append([x, y, w, h, self.id_count])
                    self.id_count += 1

        # Clean the dictionary by center points to remove IDs not used anymore
        new_center_points = {}
        for obj_bb_id in objects_bbs_ids:
            _, _, _, _, object_id = obj_bb_id
            center = self.center_points[object_id]
            new_center_points[object_id] = center

        # Update dictionary with IDs not used removed
        self.center_points = new_center_points.copy()
        return objects_bbs_ids

model = YOLO('yolov9c.pt')

class_list = ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush']

tracker = Tracker()
count = 0

cap = cv2.VideoCapture('trial2.mp4') 

down = {}
counter_down = set()
while True:    
    ret, frame = cap.read()
    if not ret:
        break
    count += 1

    results = model.predict(frame)

    a = results[0].boxes.data
    a = a.detach().cpu().numpy()  
    px = pd.DataFrame(a).astype("float")
    
    list = []
             
    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        d = int(row[5])
        c = class_list[d]
        if 'car' in c:
            list.append([x1, y1, x2, y2])

    bbox_id = tracker.update(list)
    
    for bbox in bbox_id:
        x1, y1, x2, y2, id = bbox
        cx = (x1 + x2) // 2
        cy = (y1 + y2) // 2
        cv2.circle(frame, (cx, cy), 4, (0, 0, 255), -1)  # Draw center points of bounding box
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Draw bounding box
        cv2.putText(frame, str(id), (cx, cy), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 255), 2)

        y = 308
        offset = 7

        ''' condition for red line '''
        if y < (cy + offset) and y > (cy - offset):
            down[id] = cy   # Save the IDs of the cars which are touching the red line first
            counter_down.add(id) 

    text_color = (255, 255, 255)  # White color for text
    red_color = (0, 0, 255)  # (B, G, R)   

    cv2.line(frame, (282, 308), (1004, 308), red_color, 3)  # Draw red line
    cv2.putText(frame, 'red line', (280, 308), cv2.FONT_HERSHEY_SIMPLEX, 0.5, text_color, 1, cv2.LINE_AA) 

    downwards = len(counter_down)
    cv2.putText(frame, 'going down - ' + str(downwards), (60, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, red_color, 1, cv2.LINE_AA) 

    cv2.imshow("frames", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()



0: 384x640 4 persons, 12 cars, 2 motorcycles, 4 buss, 12 trucks, 1 umbrella, 2198.4ms
Speed: 15.0ms preprocess, 2198.4ms inference, 23.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 12 cars, 2 motorcycles, 4 buss, 12 trucks, 1 umbrella, 1986.1ms
Speed: 17.5ms preprocess, 1986.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 persons, 11 cars, 3 motorcycles, 6 buss, 11 trucks, 1 umbrella, 1226.9ms
Speed: 9.0ms preprocess, 1226.9ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 12 cars, 2 motorcycles, 3 buss, 13 trucks, 1 umbrella, 585.0ms
Speed: 4.0ms preprocess, 585.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 11 cars, 2 motorcycles, 3 buss, 13 trucks, 1 umbrella, 582.3ms
Speed: 0.0ms preprocess, 582.3ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 persons, 12 cars, 2 motorcycles, 4 buss, 12 trucks, 1 umb


0: 384x640 2 persons, 17 cars, 1 motorcycle, 2 buss, 10 trucks, 1 umbrella, 688.4ms
Speed: 0.0ms preprocess, 688.4ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 18 cars, 2 motorcycles, 4 buss, 9 trucks, 1 umbrella, 640.6ms
Speed: 0.0ms preprocess, 640.6ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 17 cars, 1 motorcycle, 5 buss, 8 trucks, 1 umbrella, 562.0ms
Speed: 2.1ms preprocess, 562.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 17 cars, 1 motorcycle, 5 buss, 9 trucks, 1 umbrella, 672.5ms
Speed: 0.0ms preprocess, 672.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 17 cars, 1 motorcycle, 4 buss, 8 trucks, 1 umbrella, 609.5ms
Speed: 15.6ms preprocess, 609.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 17 cars, 1 motorcycle, 2 buss, 10 trucks, 1 umbrella, 632.4ms
Speed: 